In [2]:
import netCDF4 as nc
import numpy as np

In [23]:
### Initial Conditions
coords = np.array([[0.75,0.25,0.25],
                   [1.0,0.0,0.0],
                   [1.0,1.0,0.0],
                   [0.0,1.0,0.0],
                   [0.0,0.0,1.0],
                   [1.0,0.0,1.0],
                   [1.0,1.0,1.0],
                   [0.0,1.0,1.0]])
visc = .0001
tvisc = 0.3
vel = np.array([-0.139725, 0.5442336, 0.0]) #np.array([-0.18141120257952625,-0.3185887974204738+0.5,0.0])
avgVel = np.array([0.5, 0.5, 0.5])
avgDensity = 1.0
avgResAdeq = 0.7
minDist = 0.7
metric = np.array([[0.2,0.2,0.2],[0.2,0.2,0.2],[0.2,0.2,0.2]])
density = 1.0
tke = 2.12533 #2.1627161093038669
tdr = 2.02826 #2.0366911864642083
alpha = 0.562663 #0.581358
time = 1.0 - 440.0
dt = 0.1

In [26]:
V2F_C_MU = 0.22
V2F_CETA = 70.00
V2F_CT = 6.00
BL_KOL = 1.0
BL_T = 1.0
FORCING_CL = 4.0
FORCING_FACTOR = 8.0
d_plus = minDist/visc
f_mu = 1.0 - np.exp(-0.0115*d_plus)
AF_LPX = 3.14159265359
AF_LPY = 0.25
AF_LPZ = 1.1780972450961724

print coords[0], visc, tvisc, density, tke, tdr, alpha, minDist, avgVel[0], avgVel[1], vel[0]-avgVel[0], vel[1]-avgVel[1]

c_k = tke
#c_v2 = np.minimum(0.09*f_mu/V2F_C_MU, 2.0/3.0)*np.maximum(c_k,0.0)
c_v2 = tvisc*tdr / (0.22*np.maximum(c_k,0.0))
c_d = tdr
c_a = alpha

length = FORCING_CL*(c_a*c_k)**1.5/c_d
length = np.maximum(length, V2F_CETA*(visc**(0.75))/(c_d**(0.25)) )
T_alpha = c_a*c_k/c_d
T_alpha = np.maximum(T_alpha,V2F_CT*np.sqrt(visc/c_d))
T_alpha = BL_T*T_alpha
T_kol = np.sqrt(visc/c_d)
zeta_lcl = 1.5*c_v2/c_k

xp = coords[0] + time*avgVel
lengthY = np.minimum(length, minDist)
ax = np.pi/(AF_LPX/float(np.rint((AF_LPX)/np.minimum(np.maximum(length,2.0*metric[0][0]),AF_LPX))))
ay = np.pi/(AF_LPY/float(np.rint((AF_LPY)/np.minimum(np.maximum(lengthY,2.0*metric[1][1]),AF_LPY))))
az = np.pi/(AF_LPZ/float(np.rint((AF_LPZ)/np.minimum(np.maximum(length,2.0*metric[2][2]),AF_LPZ))))

g1 =  1.0 * np.cos(ax*xp[0]) * np.sin(ay*xp[1]) * np.sin(az*xp[2])
g2 = -3.0 * np.sin(ax*xp[0]) * np.cos(ay*xp[1]) * np.sin(az*xp[2])
g3 =  2.0 * np.sin(ax*xp[0]) * np.sin(ay*xp[1]) * np.cos(az*xp[2])

g_inst = np.zeros(3)
g_inst[0] = g1/3.0
g_inst[1] = g2/3.0
g_inst[2] = g3/3.0

print g_inst

u_prime = vel - avgVel

F_target = FORCING_FACTOR * np.sqrt(c_a*c_v2)/T_alpha

prod_r = np.dot((F_target*dt)*g_inst, u_prime)
print (F_target*dt)*g_inst, u_prime
arg = np.sqrt(avgResAdeq) - 1.0
if(arg < 0.0): 
    arg = 1.0 - 1.0/np.sqrt(avgResAdeq)
a_sign = np.tanh(arg)

Sa = a_sign
a_kol = np.minimum(BL_KOL*np.sqrt(visc*c_d)/(c_k),1.0)

if (a_sign < 0.0):
    if( c_a <= a_kol):
        Sa = Sa - (1.0+a_kol-c_a)*a_sign    
else:
    if(c_a >= 1.0):
        Sa = Sa - c_a*a_sign

prod_temp = (c_a*c_k/T_alpha)*Sa

fd_temp = avgResAdeq

if(fd_temp < 1.0 and prod_r >= 0.0):
    C_F = -1.0 * F_target * Sa
else:
    C_F = 0.0
    
norm = C_F
print norm, fd_temp, prod_r, F_target

g_inst[0] = norm*g_inst[0]
g_inst[1] = norm*g_inst[1]
g_inst[2] = norm*g_inst[2]


[0.75 0.25 0.25] 0.0001 0.3 1.0 2.12533 2.02826 0.562663 0.7 0.5 0.5 -0.639725 0.044233599999999984
[ 7.99870841e-16 -2.97651445e-01 -1.07508594e-14]
[ 9.28711536e-16 -3.45596210e-01 -1.24825742e-14] [-0.639725   0.0442336 -0.5      ]
0.0 0.7 -0.015286964507286102 11.610768746505324


In [27]:
g_inst

array([ 0., -0., -0.])

In [72]:
xp

array([-219.25, -219.25, -219.25])

In [73]:
ay

12.566370614359172

In [74]:
coords[0]

array([0.25, 0.25, 0.25])

In [75]:
az

2.6666666666666665

In [ ]:
time_temp = c_k/c_d
time_des(icv) = MAX(time_temp,V2F_CT*sqrt(constant_nu_value/c_d))
time_des(icv) = MIN(time_des(icv), V2F_ALPHA * v2f_tke(icv) / max(sqrt(6.0_WP)*V2F_C_MU*v2f_v2(icv)*strain,1.0E-12_WP) )
